In [ ]:
%%html
<style>
.prompt_container { display: none !important; }
.prompt { display: none !important; }
.run_this_cell { display: none !important; }

.slides {
    position: absolute;
    top: 0;
    left: 0;
}
</style>

# ProgRes, Part IIb

# Web servers

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

# Roadmap

- Part I: done
- Part II (Web services)
  - Last week: Client side
  - This week: Server side
    - Bottle
    - Testing
    - Web Server + API
- Part III: P2P

# Methodology

- Course and practicals are made on notebooks (jupyter or jupyterlab)
- This means you will send your practical notebooks. Please put your name on the file and inside as well!
- Practicals: for some advanced optional questions, you may add some traditional `.py` files (companion packages) -> `zip`
- Mini-projects: `zip` with a mix of notebooks and `.py` files is expected. Report can be integrated inside a notebook (preferred) or a PDF, limited to 10 pages.

# Jupyter notebook?

A notebook is just a text file with extension `.ipynb` that contains cells.
- Two main types of cells:
  - Markdown cells to write formatted text. You can itemize or write maths like $\frac{\sqrt{\pi}}{2}$
  - Code cell to execute Python code
- This is a markdown cell

In [ ]:
# This is a code cell
x = 1+1

In [ ]:
# Cell codes share the same workspace
x

# Using Jupyter Notebook

Two modes:
- Command mode (blue). Hit `esc` to enter it
- Edit mode (green). Hit `enter` on a cell to edit
- There are many shortcuts (hit `H` on command mode to see them)

## Create an API in Python

# Reminder

HTTP can be used to communicate:
- Between the server and the user (frontend)
- Internally, between the server and other services (backend)

API (Application Programming Interface) is a set of exposed methods for interaction between programms.

REST is a *code of conduct* for http-based API.

# Web libraries

Many libraries / framework exist in Python.
From https://www.educative.io/blog/top-python-web-frameworks

1. Flask: light, yet powerful. Adapted to middle-complexity websites.
2. Django: ultra-powerful, ultra-complete. Adapted to production-ready complex websites.
3. FastAPI: light, robust, easy-to-learn, production-ready.
4. CherryPy: Flask simpler and more pythonic. Adapted to professors.
5. Bottle: very light, standalone. Adapted to courses and mini-projects.


# The bottle framework

- **Routing**: Requests to function-call mapping with support for clean and dynamic URLs.
- **Templates**: Fast and pythonic built-in template engine.
- **Utilities**: Convenient access to form data, file uploads, cookies, headers and other HTTP-related metadata.
- **Server**: Built-in HTTP development server and support for other WSGI capable HTTP server.

WSGI is the Web Server Gateway Interface (allows to combine multiple frameworks together).

# Hello world example

In [ ]:
from bottle import route, run
@route('/hello/<name>')
def index(name):
    return 'Hello '+name
run(host='localhost', port=8080)

# Intermezzo: Jupyter/IPython magics 

- IPython Magics are powerful commands that change the behavior of a code cell.
- Here we will use some to avoid switching back and forth between the notebook and command line.
- Using magics is never mandatory but often helpful
- If you want to learn more about magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html

# Hello world example with magics

In [ ]:
%%writefile run.py
from bottle import route, run
@route('/hello/<name>')
def index(name):
    return f'Hello {name}'
run(host='localhost', port=8080)

In [ ]:
!wt python run.py

Check that the server works: http://localhost:8080/hello/Students

# Hello world example a bit cleaner

In [ ]:
%%writefile app.py
from bottle import Bottle
app = Bottle()
@app.route('/')
def alive():
    res = ["Server running. Available routes:<br>"]
    return res+[(f"<a href='http://localhost:8080{r.rule}'>"
                 f"{r.rule}</a><br>") for r in app.routes]

In [ ]:
%%writefile run.py
from app import app
@app.route('/hello/<name>')
def index(name):
    return f'Hello {name}'
app.run()

In [ ]:
!wt python run.py

# Filters: :int, :float, :path, :re

In [ ]:
%%writefile run.py
from app import app
from bottle import static_file
@app.route('/square/<i:int>')
def square(i):
    return f'The square of {i} is {i**2}.'
@app.route('/open/<filepath:path>')
def open_file(filepath):
    return static_file(filepath, root='.')
@app.route('/slug/<id_slug:re:[0-9]+-.*>')
def slug(id_slug):
    id_slug = id_slug.split('-', 1)
    return f'Slug {id_slug[1]} has id {id_slug[0]}.'
app.run()

In [ ]:
!wt python run.py

# Specify the method (GET, POST, ...)

In [ ]:
%%writefile run.py
from app import app
from bottle import static_file
@app.route('/square/<i:int>', method='get')
def square(i):
    return f'The square of {i} is {i**2}.'
@app.get('/open/<filepath:path>') # Equivalent
def open_file(filepath):
    return static_file(filepath, root='.')
@app.get('/slug/<id_slug:re:[0-9]+-.*>')
def slug(id_slug):
    id_slug = id_slug.split('-', 1)
    return f'Slug {id_slug[1]} has id {id_slug[0]}.'
app.run()

In [ ]:
!wt python run.py

# A route can return...

In [ ]:
%%writefile run.py
from app import app
import json
from bottle import static_file
@app.get('/nothing')
def nothing():
    return None
@app.get('/string')
def string():
    return json.dumps({'a': 42, 'b': None})
@app.get('/file')
def file():
    return static_file('run.py', root='.')
@app.get('/iterator')
def iterator():
    return ['Line 1\n', 'Line 2 \n']
app.run()

In [ ]:
!wt python run.py

# The response object

In [ ]:
%%writefile run.py
from app import app
import json
from bottle import response
@app.route('/powers/<i:int>')
def powers(i):
    response.content_type = 'application/json; charset=utf-8'
    return json.dumps({'i':i, 'square': i**2, 'cube': i**3})
app.run()

In [ ]:
!wt python run.py

- When returning a string, it is encoded with respect to the given content-type (only bytes are sent back to the client)
- Here, the server sends a byte array encoded in utf-8, that represents an object formatted in JSON.

# Sending Errors, Redirects

In [ ]:
%%writefile run.py
from app import app
from bottle import abort, redirect, static_file
@app.route('/private/<id:int>')
def private(id):
    abort(401, "Sorry, access denied.")
@app.route('/open/<filepath:path>')
def open_file(filepath):
    return static_file(filepath, root='.')
@app.route('/json/<stem>')
def getjson(stem):
    redirect(f"/open/{stem}.json")
app.run()

In [ ]:
!wt python run.py

## Testing your code

# Why testing is important?

- Domino error
  - A function `two` returns 3 instead of 2
  - In a far, far away submodule, lies `4**two()`
- Coverage:
  - Are you sure you don't have dead code lying?
  - https://balouf.github.io/stochastic_matching/readme.html
- Quick reaction:
  - When you break things
  - When another dev breaks things
  - When the new release of an obscure package breaks things 

# How to test?

- unittest
  - The original testing framework
  - You should not use it (too heavy!)
- pytest
  - like unittest but simpler
  - You should use it!
- doctests (not covered here but nice!)
  
 https://www.lincs.fr/events/testing-in-python/

# pytest syntax

If it starts with `test`, it's a test.

In [ ]:
%%writefile test_file.py

def square(x):
    return x**2

def test_square():
    assert square(2)==4

In [ ]:
!pytest

# Example: arithmetic testing

In [ ]:
%%writefile run_maths.py
from app import app
from json import dumps
@app.route("/add/<i:int>/<j:int>")
def add(i,j):
    return dumps([i+j])
@app.route("/sub/<i:int>/<j:int>")
def sub(i,j):
    return dumps([i-j])
@app.route("/mul/<i:int>/<j:int>")
def mul(i,j):
    return dumps([i*j])
@app.route("/div/<i:int>/<j:int>")
def div(i,j):
    return dumps([i//j,i%j])
app.run(host='localhost', port=8070)

In [ ]:
!wt python run_maths.py

# Example: arithmetic testing

In [ ]:
from requests import get
server_ip = "127.0.0.1"
server_port = 8070
r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
print(r1.text)
r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
print(r2.text)

# Example: arithmetic testing

In [ ]:
%%writefile test_file.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 8070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]

In [ ]:
!pytest

# Example: arithmetic testing

In [ ]:
%%writefile test_file.py
from requests import get
from json import loads
server_ip = "127.0.0.1"
server_port = 8070
def test_add():
    r1 = get(f"http://{server_ip}:{server_port}/add/4/5")
    assert loads(r1.text) == [ 9 ]
    r2 = get(f"http://{server_ip}:{server_port}/add/7/14")
    assert loads(r2.text) == [21]
def test_sub():
    r1 = get(f"http://{server_ip}:{server_port}/sub/4/5")
    assert loads(r1.text) == [ 1 ]
    r2 = get(f"http://{server_ip}:{server_port}/sub/2/2")
    assert loads(r2.text) == [0]

In [ ]:
!pytest

# Example: strings

In [ ]:
%%writefile run_string.py
from app import app
@app.route("/to_upper/<s>")
def to_upper(s):
    return s.upper()
app.run(host='localhost', port=8060)

In [ ]:
!wt python run_string.py

# Example: strings

In [ ]:
%%writefile test_file.py
from requests import get
server_ip = "127.0.0.1"
server_port = 8060
def test_upper():
    r1 = get(f"http://{server_ip}:{server_port}/to_upper/LaTeX")
    assert r1.text == 'LATEX'
    r2 = get(f"http://{server_ip}:{server_port}/to_upper/Été")
    assert r2.text == 'ÉTÉ'

In [ ]:
!pytest

## API Hour


# A First micro-service

In [ ]:
%%writefile run.py
ip="127.0.0.1"
math_port=8070
from app import app
from bottle import request
from requests import get
from json import loads
form = """<form action="/" method="post">
 i: <input name="i" type="text" />
 j: <input name="j" type="text" />
<input value="Add" type="submit" />
</form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    i = request.forms['i']
    j = request.forms['j']
    r = get(f"http://{ip}:{math_port}/add/{i}/{j}")
    l = loads(r.text)
    return f"<h1>{i}+{j}={l[0]}</h1>"    
app.run()

In [ ]:
!wt python run.py

# What happened?

- User sends a get method to 8080
- 8080 sends a html form to user
- User sends a post method to 8080
- 8080 sends a get method to 8070
- 8070 sends a json to 8080
- 8080 sends a html title to user

# Another one?

In [ ]:
%%writefile run.py
ip="127.0.0.1"
string_port=8060
from bottle import request, Bottle
from requests import get
app = Bottle()
form = """<form action="/" method="post">
 Word: <input name="s" type="text" />
 <input value="To Uppercase" type="submit" />
 </form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    s = request.forms['s'] # unicode is universal?
    r = get(f"http://{ip}:{string_port}/to_upper/{s}")
    l = r.text
    return f"<h1>{s}.upper()={l}</h1>"    
app.run()

In [ ]:
!wt python run.py

# Getting parameters:

- from the URL, e.g. `/entry?order=name`: `request.query['order']`
- from the post data, e.g. a posted form: `request.forms['order']`
- from URL or post data: `request.params['order']`
- Retrieve a posted file:
  - `image = request.files['image']`
  - `image.save('images/uploaded')`

# A bit of practice

- Here we used different port numbers to demonstrate interactions between several machine
- In practice, if the machines are distinct, you use the same port
- Multiple services can run on the same machine
  - One developper writes the maths routes
  - One developper writes the form routes
  - One developper writes the server
- Good practice: don't use http for internal calls
- But sometimes, it's better than running X servers

# First example revisited: the backend

In [ ]:
%%writefile app_maths.py
from bottle import Bottle
from json import dumps
app = Bottle()
@app.route("/add/<i:int>/<j:int>")
def add(i,j):
    return dumps([i+j])
@app.route("/sub/<i:int>/<j:int>")
def sub(i,j):
    return dumps([i-j])
@app.route("/mul/<i:int>/<j:int>")
def mul(i,j):
    return dumps([i*j])
@app.route("/div/<i:int>/<j:int>")
def div(i,j):
    return dumps([i//j,i%j])

# First example revisited: the frontend

In [ ]:
%%writefile app_user.py
from bottle import Bottle, request
from json import loads
import requests
app = Bottle()
form = """<form action="/" method="post">
 i: <input name="i" type="text" />
 j: <input name="j" type="text" />
<input value="Add" type="submit" />
</form>"""
@app.get("/")
def input_form():
    return form
@app.post("/")
def process_form():
    i = request.params['i']
    j = request.params['j']
    r = requests.get(f"http://127.0.0.1:8080/maths/add/{i}/{j}")
    l = loads(r.text)
    return f"<h1>{i}+{j}={l[0]}</h1>"

# First example revisited: the server

In [ ]:
%%writefile run.py
from gevent import monkey; monkey.patch_all()
from bottle import Bottle
from app_user import app as app_user
from app_maths import app as app_maths
app = Bottle()
app.mount('/maths', app_maths)
app.merge(app_user)
app.run(server='gevent')

In [ ]:
!wt python run.py

## Templates

# Templates

- Writing nice html requires lot of boring text (tags, etc...)
- With templates, you can prepare skeletons with placeholders to save times
- Many possibilities:
  - Native Python: f-strings, % substitution, string.Template
  - bottle provides its own template factory

# Basic templates

In [ ]:
from bottle import SimpleTemplate, template
tpl = SimpleTemplate('Hello {{name}}!')
s = tpl.render(name='World')
print(s)
my_dict={'number': '123', 'street': 'Fake St.', 'city': 'Fakeville'}
s = template('The address is at {{number}} {{street}}, {{city}}', my_dict)
print(s)

# If condition

In [ ]:
tpl = 'Hello {{name if name != "World" else "Planet"}}!'
s = template(tpl, name='World')
print(s)
s = template(tpl, name='Sébastien')
print(s)

# Warning: HTML entities

In [ ]:
tpl = 'Hello {{name}}!'
s = template(tpl, name='<b>World</b>')
print(s)
tpl = 'Hello {{!name}}!'
s = template(tpl, name='<b>World</b>')
print(s)

# Template files

In [ ]:
%%writefile simple.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
 {{content}}
 </body>
</html

# Template files

In [ ]:
from IPython.display import HTML
site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple.tpl',site)
print(s)
HTML(s)

# Conditions in template files

In [ ]:
%%writefile simple_if.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1>
 % if bold_content:
 <b>
 % end
 {{content}}
 % if bold_content:
 </b>
 % end
 </body>
</html>

In [ ]:
site = { 'title': 'This is the title', 
'content': 'This is the content'}
s = template('simple_if.tpl',site, bold_content=True)
HTML(s)

# Template files loops

In [ ]:
%%writefile simple_for.tpl
<html>
 <head><title>{{title}}</title></head>
 <body>
 <h1>{{title}}</h1><ul>
 % for content in contents:
 % if bold:
 <li><b>{{content}}</b></li>
 % else:
 <li>{{content}}</li>
 % end
 %end
 </ul></body>
</html>

In [ ]:
site = { 'title': 'This is the title', 'contents': 
        ['This is the first line of content', 'second line', 'third line']}
s = template('simple_for.tpl', site, bold=False)
HTML(s)

# The end!